<a href="https://colab.research.google.com/github/ghinwa-mj/WBWork_sharing/blob/main/Sharable_Copy_of_Serbia_AutomatedTrendAnalysis_vf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

import os

from google.colab import drive
drive.mount('/content/drive')

google_drive_root = '/content/drive/MyDrive'

# Define the path to the Google Drive folder
google_drive_folder = '/content/drive/MyDrive/World Bank/WB_PimPam'

# Change the working directory to the specified folder
os.chdir(google_drive_folder)

In [ ]:
pip install langchain openai tiktoken faiss-cpu pypdf langchain_openai langchain_experimental

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.8/812.8 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 266.9/266.9 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 40.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 286.1/286.1 kB 31.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 67.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.6/276.6 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [ ]:
#Libraries used for OpenAI
from langchain.schema import SystemMessage, HumanMessage
from langchain.callbacks import get_openai_callback

#openai_api_key = "xxx"

from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain.chat_models import ChatOpenAI
from langchain.agents.agent_types import AgentType
from langchain.llms import OpenAI

In [ ]:
#1.Read the DataFrame
#2. Adjustements to the DF necessary for the analysis

df = pd.read_csv("GDD_Data/WB_GPBP_GED_SRB_2_municipality.csv")

#Change the naming of District and Municipality (Serbia Specific)
df = df.rename(columns = {'DISTRICT': 'MUNICIPALITY',
                          'PROVINCE': 'DISTRICT'})

df['econ_score_rank'] = df.groupby('YEAR')['ECON_SCORE'].rank(ascending=True)
# Calculate the percentile score
df['NATIONAL_RANKING_ECONOMICS'] = df.groupby('YEAR')['econ_score_rank'].transform(lambda x: (x.rank() - 1) / len(x) * 100)
# Round the percentile score to make it more readable
df['NATIONAL_RANKING_ECONOMICS'] = df['NATIONAL_RANKING_ECONOMICS'].round(2)

del df['econ_score_rank']

df['env_score_rank'] = df.groupby('YEAR')['ENVR_SCORE'].rank(ascending=True)
# Calculate the percentile score
df['NATIONAL_RANKING_ENVIRONMENTAL'] = df.groupby('YEAR')['env_score_rank'].transform(lambda x: (x.rank() - 1) / len(x) * 100)
# Round the percentile score to make it more readable
df['NATIONAL_RANKING_ENVIRONMENTAL'] = df['NATIONAL_RANKING_ENVIRONMENTAL'].round(2)

del df['env_score_rank']

# Calculate the percentile score
df['DISTRICT_RANKING_ECONOMICS'] = df.groupby(['YEAR', 'DISTRICT'])['ECON_SCORE'].transform(lambda x: (x.rank() - 1) / len(x) * 100)
# Round the percentile score to make it more readable
df['DISTRICT_RANKING_ECONOMICS'] = df['DISTRICT_RANKING_ECONOMICS'].round(2)

# Calculate the percentile score
df['DISTRICT_RANKING_ENVIRONMENTAL'] = df.groupby(['YEAR', 'DISTRICT'])['ENVR_SCORE'].transform(lambda x: (x.rank() - 1) / len(x) * 100)
# Round the percentile score to make it more readable
df['DISTRICT_RANKING_ENVIRONMENTAL'] = df['DISTRICT_RANKING_ENVIRONMENTAL'].round(2)
#df.head(10)

In [ ]:
#Initiate Agent Connexting the DF to the LLM
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0),
    df, verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS)

#LLM for Q&A
llm_talkative = ChatOpenAI(temperature=0,
                       model = "gpt-3.5-turbo",
                           max_tokens= 1000)

In [ ]:
def rankings_analysis(municipality):

  most_recent_year = df['YEAR'].max()
  system_prompt = "You are an economist. You will extract the economic and environmental data needed to answer questions. You will always provide answers in full sentence format. You are concise and straight to the point."

  #Extract the Overall Economic Ranking Percentile
  input = "What is the 'NATIONAL_RANKING_ECONOMICS' indicator for the municipality of " +str(municipality) + " in 2022."
  overall_ranking_econ = agent.run(input)
  overall_ranking_econ = overall_ranking_econ.replace("NATIONAL_RANKING_ECONOMICS", "national ranking of the economic score compared to all municipalities in the country")
  overall_ranking_econ = overall_ranking_econ[:-1] + '%' + overall_ranking_econ[-1]

  #Extract the Overall Environmental Ranking Percentile
  environmental_ranking_input = "What is the 'NATIONAL_RANKING_ENVIRONMENTAL' indicator for the municipality of " + str(municipality) +" in " + str(most_recent_year) + "."
  percentile_ranking_env = agent.run(environmental_ranking_input)
  percentile_ranking_env = percentile_ranking_env.replace("NATIONAL_RANKING_ENVIRONMENTAL", "national ranking of the environmental score compared to all municipalities in the country")
  percentile_ranking_env = percentile_ranking_env[:-1] + '%' + percentile_ranking_env[-1]


  #Extract the provincial Economic Ranking Percentile
  input = "What is the 'DISTRICT_RANKING_ECONOMICS' indicator for the municipality of " +str(municipality) + " in 2022."
  provincial_ranking_econ = agent.run(input)
  provincial_ranking_econ = provincial_ranking_econ.replace("DISTRICT_RANKING_ECONOMICS", "district ranking of the economic score compared to all municipalities in the same district")
  provincial_ranking_econ = provincial_ranking_econ[:-1] + '%' + provincial_ranking_econ[-1]


  #Extract the provincial Environmental Ranking Percentile
  environmental_ranking_input = "What is the 'DISTRICT_RANKING_ENVIRONMENTAL' indicator for the municipality of " + str(municipality) +" in " + str(most_recent_year) + "."
  provincial_ranking_env = agent.run(environmental_ranking_input)
  provincial_ranking_env = provincial_ranking_env.replace("DISTRICT_RANKING_ENVIRONMENTAL", "district ranking of the environmental score compared to all municipalities in the same district")
  provincial_ranking_env = provincial_ranking_env[:-1] + '%' + provincial_ranking_env[-1]


  #Prompt the LLM --> Explains the scores and their significance
  human_message = str(provincial_ranking_econ) + str(overall_ranking_econ) + str(provincial_ranking_env) + str(percentile_ranking_env) + " Summarize the results of these indicators for this specific year in a few sentences always specify whether the ranking is national or district. Is the municipality a strong performer on the district level or on the country level? Does the municipality perform better economically or environmentally or both? What is the overall implication?"

  messages = [
        SystemMessage(content=system_prompt),
        HumanMessage(content= human_message)
    ]

  response_rankings = llm_talkative(messages)

  return "In their current format the indicators allow us to understand " + str(municipality) + "'s performance compared to other municipalities in the same district and all over the country. \n" + response_rankings.content

In [ ]:
def envr_trend_analysis(municipality):

  system_prompt = "You are an economist. You will extract the economic and environmental data needed to answer questions. You will always provide answers in full sentence format. You are concise and straight to the point."
  most_recent_year = df['YEAR'].max()


  #Extract the components of the environmental score
  env_explanation = "The environmental score is based on the three indicators of: Air Quality, Weather and Green Space. Summarize how these three indicators are changing each year for the municipality of " +str(municipality) + "?"
  env_explanation = agent.run(env_explanation)

  indicators_analysis_prompt = str(env_explanation) + " Remember that higher scores are better. In two or three sentences, summarize the overall trend of the three indicators throughout time while mentioning whether the final score is high or low. Did any indicator show huge fluctuations? Which indicators have shown an overall increase and which others have shown an overall decrease? Conclude briefly."

  #Prompt the LLM --> Explains the scores and their significance

  indicators_trend = [
      SystemMessage(content=system_prompt),
      HumanMessage(content=indicators_analysis_prompt)
      ]

  response_indicators = llm_talkative(indicators_trend)

  return "The environmental score is based on the three indicators of: Air Quality, Weather and Green Space. " + response_indicators.content

In [ ]:
def green_built_space(municipality):
  system_prompt = "You are an economist. You will answer questions to provide economic and environmental analysis in a professional, scientific, concise and consistent manner. You will be concise and straight to the point. You will not be repetitive."
  input = "The indicator 'BUILD_PCT' represents the percentage of built space in a given region. The indicator 'GREEN_PCT' represents the percentage of green space in a given region. Summarize in one sentence how both the green and the built space are increasing or decreasing over time in the municipality of " + str(municipality) + ". Pay attention to where the change is substantial or not, is the percentage increase too large or may be natural? In one sentence, explain the economic or environmental implication of these results."

  response = agent.run(input)
  return response

In [ ]:
### Attempt on a limited set of Municipalities ###

#df[df["MUNICIPALITY"] == "VelikoGradište"]
#list_output = list()

municipality_list = df[df["DISTRICT"] == "Braničevski"]["MUNICIPALITY"].unique()
#municipality_list

array(['Golubac', 'Kučevo', 'MaloCrniće', 'Petrovac', 'Požarevac',
       'VelikoGradište', 'Žabari', 'Žagubica'], dtype=object)

In [ ]:
for municipality_temp in municipality_list:

  envr_trend = envr_trend_analysis(municipality_temp)
  ranking = rankings_analysis(municipality_temp)
  green_built = green_built_space(municipality_temp)

  trends = str(ranking) + "\n" + str(envr_trend) + "\n" + str(green_built)
  list_output.append([municipality_temp, trends])



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['MUNICIPALITY']=='Golubac'][['YEAR','AIR_SCORE','TEMP_SCORE','FOREST_SCORE']]"}`


     YEAR  AIR_SCORE  TEMP_SCORE  FOREST_SCORE
224  2019  98.136646   11.801242     22.360248
225  2020  33.540373   81.987578     31.677019
226  2021  89.440994   23.602484     12.422360
227  2022  49.068323   85.714286     39.751553For the municipality of Golubac, the environmental indicators are changing each year as follows:

- **Air Quality Score**:
  - 2019: 98.14
  - 2020: 33.54
  - 2021: 89.44
  - 2022: 49.07

- **Weather Score**:
  - 2019: 11.80
  - 2020: 81.99
  - 2021: 23.60
  - 2022: 85.71

- **Green Space Score**:
  - 2019: 22.36
  - 2020: 31.68
  - 2021: 12.42
  - 2022: 39.75

These scores indicate the environmental performance in terms of air quality, weather conditions, and green space for the municipality of Golubac over the years.

> Finished chain.


> Entering new AgentExecutor chain...

Invoki

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


The 'NATIONAL_RANKING_ECONOMICS' indicator for the municipality of MaloCrniće in 2022 is 66.77.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['MUNICIPALITY']=='MaloCrniće'][df['YEAR']==2022]['NATIONAL_RANKING_ENVIRONMENTAL']"}`


235    40.68
Name: NATIONAL_RANKING_ENVIRONMENTAL, dtype: float64

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


The 'NATIONAL_RANKING_ENVIRONMENTAL' indicator for the municipality of MaloCrniće in 2022 is 40.68.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['MUNICIPALITY']=='MaloCrniće'][df['YEAR']==2022]['DISTRICT_RANKING_ECONOMICS']"}`


235    50.0
Name: DISTRICT_RANKING_ECONOMICS, dtype: float64

...

The 'NATIONAL_RANKING_ECONOMICS' indicator for the municipality of Žagubica in 2022 is 17.39.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['MUNICIPALITY']=='Žagubica'][df['YEAR']==2022]['NATIONAL_RANKING_ENVIRONMENTAL'].values[0]"}`


93.48

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


The 'NATIONAL_RANKING_ENVIRONMENTAL' indicator for the municipality of Žagubica in 2022 is 93.48.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[(df['MUNICIPALITY']=='Žagubica') & (df['YEAR']==2022)]['DISTRICT_RANKING_ECONOMICS'].values[0]"}`


0.0The 'DISTRICT_RANKING_ECONOMICS' indicator for the municipality of Žagubica in 2022 is 0.0.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df[df['MUNICIPALITY']=='Žagubica'][df['YEAR']==2022]['DISTRICT_RANKING_ENVIRONMENTAL']"}`


255    87.5
Name: DISTRICT_RANKING_ENVIRONMENTAL, dtype: float64

<string>:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


The 'DISTRICT_RANKING_ENVIRONMENTAL' indicator for the municipality of Žagubica in 2022 is 87.5.

> Finished chain.


> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "df_zagubica = df[df['MUNICIPALITY'] == 'Žagubica']\nsummary = df_zagubica[['YEAR', 'BUILT_PCT', 'GREEN_PCT']].groupby('YEAR').mean()\nsummary"}`


      BUILT_PCT  GREEN_PCT
YEAR                      
2019   1.275233  90.824043
2020   1.232774  92.051280
2021   1.245047  91.076098
2022   1.282779  90.791946In the municipality of Žagubica, the percentage of built space has been relatively stable around 1.25% over the years, while the percentage of green space has shown a slight decrease from 92.05% in 2020 to 90.79% in 2022. This gradual decrease in green space may indicate a natural fluctuation in the environment, but it could also have negative environmental implications such as reduced biodiversity and ecosystem services in the region.

> Finished chain.


In [ ]:
 list_output

[['Golubac',
  "In their current format the indicators allow us to understand Golubac's performance compared to other municipalities in the same district and all over the country. \nIn 2022, the municipality of Golubac ranks 12.5% in the district for economic performance and 35.09% nationally. For environmental performance, it ranks 62.5% in the district and 76.71% nationally. Golubac performs better environmentally both at the district and national levels. The municipality's environmental performance is stronger compared to its economic performance, indicating a focus on sustainability and potentially attracting eco-conscious investments.\nThe environmental score is based on the three indicators of: Air Quality, Weather and Green Space. The overall trend of the environmental indicators for Golubac municipality shows fluctuations over the years. The Air Quality Score decreased significantly from 2019 to 2020 but then increased in 2021 and decreased again in 2022. The Weather Score and 

In [ ]:
import json

df_output = pd.DataFrame(list_output, columns=['MUNICIPALITY', 'TRENDS'])
json_data = json.dumps(list_output)

df_output.to_json('output_serbia_municipality_v3.json', orient='records', lines=True)